In [13]:
from regraph.neo4j.graphs import Neo4jGraph

In [14]:
g = Neo4jGraph("bolt://localhost:7687", "neo4j", "admin")

In [15]:
res = g.clear()

In [16]:
print(res)

In [17]:
nodes = ["a", "b", "c", "d", "e", "f"]
edges = [("a", "b"), ("d", "b"), ("a", "c"), ("e", "a"), ("f", "d")]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

MATCH (a), (d), (f), (c), (b), (e) WHERE a.id = 'a' and d.id = 'd' and f.id = 'f' and c.id = 'c' and b.id = 'b' and e.id = 'e' CREATE (a)-[:edge]->(b), (d)-[:edge]->(b), (a)-[:edge]->(c), (e)-[:edge]->(a), (f)-[:edge]->(d)


In [4]:
g.add_node("x")

In [5]:
g.add_edge("x", "c")

MATCH (c), (x) WHERE c.id = 'c' and x.id = 'x' CREATE (x)-[:edge]->(c)


In [6]:
g.remove_edge("x", "c")

In [7]:
g.nodes()

['a', 'b', 'c', 'd', 'e', 'f', 'x']

In [10]:
g.edges()

[('e', 'a'), ('a', 'b'), ('d', 'b'), ('a', 'c'), ('f', 'd')]